In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
%reload_ext autoreload

In [10]:
import detecting_fake_news.trainer
from detecting_fake_news.trainer import Trainer
from detecting_fake_news.data import get_local_data, get_cloud_data, get_preprocessed_data

## contents of trainer.py

```python
from os import path, pipe
from detecting_fake_news.data import get_local_data, get_cloud_data, get_preprocessed_data
from detecting_fake_news.preprocessing import TextPreprocessor
from detecting_fake_news.params import BUCKET_NAME, BUCKET_TRAIN_DATA_PATH, LOCAL_TRAIN_DATA_PATH, PREPROCESSED_DATA_PATH
from detecting_fake_news.gcp import storage_upload
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
import joblib
from termcolor import colored
import pandas as pd
import os




class Trainer(object):
    '''
    The Trainer class fits trains, evaluates, and saves an NLP model.
    The main method is Trainer.run.
    When instantiating an instance of the Trainer class, the user provides two
    positional arguments: X_col, and y_col.
    The user may also create an instance with data already set i.e. data=df.
    The user may also set the boolean keyword argument, preprocessed, to True if
    they are instantiating a Trainer instance using a preprocessed dataframe.
    '''
    def __init__(self, X_col, y_col, data=None, preprocessed=False):
        self.X_col = X_col
        self.y_col = y_col
        self.preprocessed = preprocessed

        self.vectorizer = TfidfVectorizer()
        self.model = MultinomialNB()
        self.pipe = None
        self.trained_model = None

        self.data = data

    def preprocess(self, new_line=True, punct=True, lower=True, accent=True,
                   numbers=True, stemm=False, lemm=True, stop_words=True):
        print("dropping rows of empty text")
        self.data = self.data.dropna(subset=[self.X_col])
        X = self.data[self.X_col]
        y = self.data[self.y_col]
        preproc = TextPreprocessor(new_line=new_line, punct=punct, lower=lower,
                                   accent=accent, numbers=numbers, stemm=stemm,
                                   lemm=lemm, stop_words=stop_words)
        print("preprocessing data with following parameters:")
        for k, v in vars(preproc).items():
            if v == True:
                print(f"{k}, ", end='')
        print('')
        X_clean = preproc.transform(X)
        self.data = pd.DataFrame({self.y_col: y, self.X_col: X_clean})
        self.preprocessed = True

    def set_pipeline(self):
        '''resets self.pipe and self.trained_model to None then sets self.pipe'''
        self.pipe = None
        self.trained_model = None
        pipe = Pipeline([('vectorizer', self.vectorizer),
                         ('model', self.model)])
        self.pipe = pipe

    def run(self):
        '''if self.data is not preprocessed, preprocesses according to params,
           sets self.data to the preprocessed data, and splits self.data into train/test;
           fits a pipeline to X_train, y_train; evaluates on X_test, y_test;
           prints an accuracy score'''
        if self.preprocessed == False:
            self.preprocess()
        X_clean = self.data[self.X_col]
        y = self.data[self.y_col]
        X_train, X_test, y_train, y_test = train_test_split(X_clean, y, test_size=0.25)
        print("setting pipeline")
        self.set_pipeline()
        print("vectorizing data and fitting model")
        self.trained_model = self.pipe.fit(X_train, y_train)
        print("evaluating on test data")
        self.evaluate(X_test, y_test)

    def evaluate(self, X_test, y_test):
        '''predicts y_pred based on X_test and scores accuracy on y_test'''
        if self.trained_model:
            y_pred = self.trained_model.predict(X_test)
            score = accuracy_score(y_test, y_pred)
            print(colored(f"model accuracy: {score}", "green"))
        else:
            print("please train a model first using Trainer.run")

    def save_model_locally(self):
        '''save self.trained_model into a .joblib format'''
        if self.trained_model:
            joblib.dump(self.trained_model, 'model.joblib')
            print(colored("model.joblib saved locally", "green"))
        else:
            print("please train a model first using Trainer.run")

    def save_preprocessed(self, data_file_name):
        '''save a .csv file of preprocessed data in detecting_fake_news/data'''
        abspath = os.path.join(PREPROCESSED_DATA_PATH, data_file_name)
        if self.preprocessed:
            self.data.to_csv(abspath, columns=[self.y_col, self.X_col], index=False)
            print(colored(f"{data_file_name} saved in {PREPROCESSED_DATA_PATH}","green"))
        else:
            print(colored(f"preprocessed = {self.preprocessed} \nplease preprocess before saving","red"))

    def load_preprocessed(self, data_file_name, nrows=None):
        '''load a .csv file of preprocessed data from detecting_fake_news/data'''
        abspath = os.path.join(PREPROCESSED_DATA_PATH, data_file_name)
        try:
            self.data = pd.read_csv(abspath, nrows=nrows)
            if nrows:
                print(colored(f"{nrows} rows loaded from {PREPROCESSED_DATA_PATH}/{data_file_name}","green"))
            else:
                print(colored(f"data loaded from {PREPROCESSED_DATA_PATH}/{data_file_name}","green"))
            self.preprocessed = True
        except FileNotFoundError:
            print(colored(f"{data_file_name} not found in {PREPROCESSED_DATA_PATH}","red"))




if __name__=='__main__':
    df = get_cloud_data(nrows=3000)
    trainer = Trainer('text', 'label' data=df)
    trainer.run()
    #trainer.save_model_locally(trainer.trained_model)
    #storage_upload('models/MultinomialNB/model.joblib', 'model.joblib')
```

## docstring and __init__ function

## workflow examples

### using raw data

In [32]:
df = get_local_data(data_file_name='train.csv', nrows=500)
df.head()

500 rows loaded from /home/henry/code/hdavis44/detecting_fake_news/raw_data/train.csv


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [35]:
# data is a keyword argument
trainer = Trainer('text', 'label')
vars(trainer)

{'X_col': 'text',
 'y_col': 'label',
 'preprocessed': False,
 'vectorizer': TfidfVectorizer(),
 'model': MultinomialNB(),
 'pipe': None,
 'trained_model': None,
 'data': None}

In [36]:
trainer.data = df
vars(trainer)

{'X_col': 'text',
 'y_col': 'label',
 'preprocessed': False,
 'vectorizer': TfidfVectorizer(),
 'model': MultinomialNB(),
 'pipe': None,
 'trained_model': None,
 'data':       id                                              title  \
 0      0  House Dem Aide: We Didn’t Even See Comey’s Let...   
 1      1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
 2      2                  Why the Truth Might Get You Fired   
 3      3  15 Civilians Killed In Single US Airstrike Hav...   
 4      4  Iranian woman jailed for fictional unpublished...   
 ..   ...                                                ...   
 495  495  Contaminated Food from China Now Entering the ...   
 496  496  Ten Famous People on What to Read This Summer ...   
 497  497  SCANDAL: EPA could have issued an emergency or...   
 498  498  Hillary Clinton KNEW 5 years ago Anthony Weine...   
 499  499  أوروبا وخيار القوة في مواجهة اللاجئين - RT Arabic   
 
                                     author  \
 0            

In [37]:
# alternatively you can specify 'data=<your_data>' at instantiation:
trainer = Trainer('text', 'label', data=df)
vars(trainer)

{'X_col': 'text',
 'y_col': 'label',
 'preprocessed': False,
 'vectorizer': TfidfVectorizer(),
 'model': MultinomialNB(),
 'pipe': None,
 'trained_model': None,
 'data':       id                                              title  \
 0      0  House Dem Aide: We Didn’t Even See Comey’s Let...   
 1      1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
 2      2                  Why the Truth Might Get You Fired   
 3      3  15 Civilians Killed In Single US Airstrike Hav...   
 4      4  Iranian woman jailed for fictional unpublished...   
 ..   ...                                                ...   
 495  495  Contaminated Food from China Now Entering the ...   
 496  496  Ten Famous People on What to Read This Summer ...   
 497  497  SCANDAL: EPA could have issued an emergency or...   
 498  498  Hillary Clinton KNEW 5 years ago Anthony Weine...   
 499  499  أوروبا وخيار القوة في مواجهة اللاجئين - RT Arabic   
 
                                     author  \
 0            

In [39]:
trainer.data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [40]:
# if you use Trainer.run on raw data, the data will be preprocessed with default parameters
# then the trainer instance will have it's pipeline set, fit, and evaluated
trainer.run()

dropping rows of empty text
preprocessing data with following parameters:
new_line, punct, lower, accent, numbers, lemm, stop_words, 
setting pipeline
vectorizing data and fitting model
evaluating on test data
model accuracy: 0.688


In [58]:
# alternatively you can call Trainer.preprocess to specify your preprocessing parameters
trainer = Trainer('text','label',data=get_local_data(data_file_name='train.csv', nrows=500))
trainer.preprocess(stemm=True, lemm=False)

500 rows loaded from /home/henry/code/hdavis44/detecting_fake_news/raw_data/train.csv
dropping rows of empty text
preprocessing data with following parameters:
new_line, punct, lower, accent, numbers, stemm, stop_words, 


In [59]:
trainer.run()

setting pipeline
vectorizing data and fitting model
evaluating on test data
model accuracy: 0.528


In [41]:
# trainer.save_model_locally() '''save self.trained_model into a .joblib format'''

In [42]:
# trainer.save_preprocessed(data_file_name) '''save a .csv file of preprocessed data in detecting_fake_news/data'''

### using preprocessed data

In [ ]:
# trainer.load_preprocessed(data_file_name, nrows=None) 
# '''load a .csv file of preprocessed data from detecting_fake_news/data'''

In [55]:
# you can instantiate the Trainer class without data and use the trainer.load_preprocessed method
trainer = Trainer('text', 'label')
trainer.load_preprocessed('experiment.csv')
print(f"preprocessed: {trainer.preprocessed}")
trainer.data.head()

data loaded from /home/henry/code/hdavis44/detecting_fake_news/detecting_fake_news/data/experiment.csv
preprocessed: True


,label,text
0,1,house dem aide even see comey letter jason cha...
1,0,ever get feeling life circle roundabout rather...
2,1,truth might get fired october tension intellig...
3,1,video civilian killed single u airstrike ident...
4,1,print iranian woman ha sentenced six year pris...


In [57]:
# alternatively you can get preprocessed data and instantiate Trainer using it

# !! if you use this method remember to set preprocessed=True or the Trainer.run function will preprocess again !!

df = get_preprocessed_data('experiment.csv')
trainer = Trainer('text', 'label', data=df, preprocessed=True)
print(f"preprocessed: {trainer.preprocessed}")
trainer.data.head()

data loaded from /home/henry/code/hdavis44/detecting_fake_news/detecting_fake_news/data/experiment.csv
preprocessed: True


,label,text
0,1,house dem aide even see comey letter jason cha...
1,0,ever get feeling life circle roundabout rather...
2,1,truth might get fired october tension intellig...
3,1,video civilian killed single u airstrike ident...
4,1,print iranian woman ha sentenced six year pris...


In [25]:
# trainer.save_model_locally() '''save self.trained_model into a .joblib format'''

In [26]:
# trainer.save_preprocessed(data_file_name) '''save a .csv file of preprocessed data in detecting_fake_news/data'''